- 처음~2020.4월 : 거리두기 시작
- 2020.5월 : 생활방역 전환
- 2020.6월~8월 : 수도권 2단계
- 2020.9월 : 2.5단계 강화
- ~~2020.9월 : 거리두기 2단계로 하향~~
- 2020.10월 : 1단계 하향
- 2020.11월 : 정밀방역으로 세분화
- 2020.12월 : 2단계 시행
- ~~2020.12월 : 2.5단계 시행~~

+ 2021.1월~2월 : 2.5단계, 조치 완화
+ ~~2021.2월 : 2단계 완화~~
+ 2021.3월~6월 : 거리두기 개편
+ 2021.07월 거리두기 4단계

- 2022.8월~4월?? : 거리두기 해제?

In [ ]:
# 2020년 4월까지

In [ ]:
# 2020년 6월에서 8월까지

In [ ]:
# 2021년 1월에서 2월까지

In [ ]:
# 2021년 3월에서 6월까지